# 第三章

## 3.1

In [22]:
from tensorflow.keras import layers

layer = layers.Dense(32, input_shape=(784,))

我们创建了一个层，只接受第一个维度大小为 784 的 2D 张量（第 0 轴是批量维度，其大小没有指定，因此可以任意取值）作为输入。这个层将返回一个张量，第一个维度的大小变成了 32。

因此，这个层后面只能连接一个接受 32 维向量作为输入的层。使用 Keras 时，你无须担心兼容性，因为向模型中添加的层都会自动匹配输入层的形状，例如下面这段代码。

In [23]:
from tensorflow.keras import models
from tensorflow.keras import layers

model = models.Sequential()
model.add(layers.Dense(32,input_shape=(784,)))
model.add(layers.Dense(32))

dense层就是全连接层:
#作为 Sequential 模型的第一层

model = Sequential()

model.add(Dense(32, input_shape=(16,)))

#现在模型就会以尺寸为 (*, 16) 的数组作为输入，
#其输出数组的尺寸为 (*, 32)

#在第一层之后，你就不再需要指定输入的尺寸了：

model.add(Dense(32))

笔记:  对于分类、回归、序列预测等常见问题，你可以遵循一些简单的指导原则来选择正确的损失函数。例如，对于二分类问题，你可以使用二元交叉熵（binary crossentropy）损失函数；对于多分类问题，可以用分类交叉熵（categorical crossentropy）损失函数；对于回归问题，可以用均方误差（mean-squared error）损失函数；对于序列学习问题，可以用联结主义时序分类（CTC，connectionist temporal classification）损失函数，等等。只有在面对真正全新的研究问题时，你才需要自主开发目标函数。

## 每一层的作用:

In [3]:
model = models.Sequential()
model.add(layers.Conv2D(32, (3, 3), activation='relu',
                        input_shape=(150, 150, 3)))
model.add(layers.MaxPooling2D((2, 2))) #在每个 MaxPooling2D 层之后，特征图的尺寸都会减半。例如，在第一个 MaxPooling2D 层之前，特征图的尺寸是 26×26，但最大池化运算将其减半为 13×13。这就是最大池化的作用：对特征图进行下采样，与步进卷积类似
model.add(layers.Conv2D(64, (3, 3), activation='relu'))
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.Conv2D(128, (3, 3), activation='relu'))
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.Conv2D(128, (3, 3), activation='relu'))
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.Flatten())  #Flatten层用来将输入“压平”，即把多维的输入一维化，常用在从卷积层到全连接层的过渡。Flatten不影响batch的大小
model.add(layers.Dropout(0.5))  #为了进一步降低过拟合，你还需要向模型中添加一个 Dropout 层，添加到密集连接分类器之前
model.add(layers.Dense(512, activation='relu'))  #全连接层
model.add(layers.Dense(1, activation='sigmoid'))  #最后我们需要用二分类,所以要用sigmoid 函数激活

model.compile(loss='binary_crossentropy',
              optimizer=optimizers.RMSprop(lr=1e-4),
              metrics=['acc'])

NameError: name 'models' is not defined

## 3.2.2

Keras 开发概述:

(1) 定义训练数据：输入张量和目标张量。

(2) 定义层组成的网络（或模型 ），将输入映射到目标。

(3) 配置学习过程：选择损失函数、优化器和需要监控的指标。

(4) 调用模型的 fit 方法在训练数据上进行迭代。

定义模型有两种方法：一种是使用 Sequential 类（仅用于层的线性堆叠，这是目前最常见的网络架构），另一种是函数式 API （functional API，用于层组成的有向无环图，让你可以构建任意形式的架构）。

前面讲过，这是一个利用 Sequential 类定义的两层模型（注意，我们向第一层传入了输入数据的预期形状）。

In [24]:
from tensorflow.keras import models
from tensorflow.keras import layers

model = models.Sequential()
model.add(layers.Dense(32, activation = 'relu', input_shape=(784,)))
model.add(layers.Dense(10, activation = 'softmax'))

一旦定义好了模型架构，使用 Sequential 模型还是函数式 API 就不重要了。接下来的步骤都是相同的。

配置学习过程是在编译这一步，你需要指定模型使用的 *优化器* 和 *损失函数*，以及 *训练过程中想要监控的指标* 。

下面是单一损失函数的例子，这也是目前最常见的。

In [25]:
from tensorflow.keras import optimizers

model.compile(optimizer=optimizers.RMSprop(lr=0.001),
             loss='mse',
             metrics=['accuracy'])

最后，学习过程就是通过 fit() 方法将输入数据的 Numpy 数组（和对应的目标数据）传入模型，这一做法与 Scikit-Learn 及其他机器学习库类似。

In [28]:
'''
model.fit(input_tensor, target_tensor, batch_size=128, epochs=10)
'''


'\nmodel.fit(input_tensor, target_tensor, batch_size=128, epochs=10)\n'

现在你可以处理关于向量数据最常见的机器学习任务了：二分类问题、多分类问题和标量回归问题。前面三节的“小结”总结了你从这些任务中学到的要点。

在将原始数据输入神经网络之前，通常需要对其进行预处理。

如果数据特征具有不同的取值范围，那么需要进行预处理，将每个特征单独缩放。

随着训练的进行，神经网络最终会过拟合，并在前所未见的数据上得到更差的结果。

如果训练数据不是很多，应该使用只有一两个隐藏层的小型网络，以避免严重的过拟合。

如果数据被分为多个类别，那么中间层过小可能会导致信息瓶颈。

回归问题使用的损失函数和评估指标都与分类问题不同。

如果要处理的数据很少，K 折验证有助于可靠地评估模型。

## 4.2

### 1.简单的留出验证

In [ ]:
num_validation_samples = 10000
　
np.random.shuffle(data)   ←------ 通常需要打乱数据
　
validation_data = data[:num_validation_samples]   ←------ 定义验证集
data = data[num_validation_samples:]
　
training_data = data[:]   ←------ 定义训练集
　
model = get_model()             （以下3行）在训练数据上训练模型，并在验证数据上评估模型
model.train(training_data)                          
validation_score = model.evaluate(validation_data)  
　
# 现在你可以调节模型、重新训练、评估，然后再次调节……
　
model = get_model()         （以下4行）一旦调节好超参数，通常就在所有非测试数据上从头开始训练最终模型
model.train(np.concatenate([training_data,       
                            validation_data]))   
test_score = model.evaluate(test_data)           


这是最简单的评估方法，但有一个缺点：如果可用的数据很少，那么可能验证集和测试集包含的样本就太少，从而无法在统计学上代表数据。这个问题很容易发现：如果在划分数据前进行不同的随机打乱，最终得到的模型性能差别很大，那么就存在这个问题。接下来会介绍 K 折验证与重复的 K 折验证，它们是解决这一问题的两种方法。

### 2.k折验证

In [ ]:
k = 4
num_validation_samples = len(data) // k
　
np.random.shuffle(data)
　
validation_scores = []
for fold in range(k):
    validation_data = data[num_validation_samples * fold:    （以下2行）选择验证数据分区
     num_validation_samples * (fold + 1)]                  
    training_data = data[:num_validation_samples * fold] +
     data[num_validation_samples * (fold + 1):]   ←------ 使用剩余数据作为训练数据。注意，+ 运算符是列表合并，不是求和
　
    model = get_model()   ←------ 创建一个全新的模型实例（未训练）
    model.train(training_data)
    validation_score = model.evaluate(validation_data)
    validation_scores.append(validation_score)
　
validation_score = np.average(validation_scores)   ←------ 最终验证分数：K 折验证分数的平均值
　
model = get_model()     （以下3行）在所有非测试数据上训练最终模型
model.train(data)                        
test_score = model.evaluate(test_data)   


K 折验证（K -fold validation）将数据划分为大小相同的 K 个分区。对于每个分区 i ，在剩余的 K - 1 个分区上训练模型，然后在分区 i 上评估模型。最终分数等于 K 个分数的平均值。对于不同的训练集 - 测试集划分，如果模型性能的变化很大，那么这种方法很有用。与留出验证一样，这种方法也需要独立的验证集进行模型校正。

### 3.带有打乱数据的重复k折验证（无代码）

如果可用的数据相对较少，而你又需要尽可能精确地评估模型，那么可以选择带有打乱数据的重复 K 折验证（iterated K -fold validation with shuffling）。我发现这种方法在 Kaggle 竞赛中特别有用。具体做法是多次使用 K 折验证，在每次将数据划分为 K 个分区之前都先将数据打乱。最终分数是每次 K 折验证分数的平均值。注意，这种方法一共要训练和评估 P ×K 个模型（P 是重复次数），计算代价很大。

## 4.4

总结一下，防止神经网络过拟合的常用方法包括：

获取更多的训练数据

减小网络容量

添加权重正则化

添加 dropout

## 4.5.1 定义问题,收集数据集

## 4.5.2 选择衡量成功的指标

对于平衡分类问题（每个类别的可能性相同），精度和接收者操作特征曲线下面积 （area under the receiver operating characteristic curve，ROC AUC）是常用的指标。

对于类别不平衡的问题，你可以使用准确率和召回率。对于排序问题或多标签分类，你可以使用平均准确率均值（mean average precision）

## 4.5.3 确定评估方法

1:`留出验证集` 。数据量很大时可以采用这种方法。

2:`K 折交叉验证` 。如果留出验证的样本量太少，无法保证可靠性，那么应该选择这种方法。

3:`重复的 K 折验证` 。如果可用的数据很少，同时模型评估又需要非常准确，那么应该使用这种方法。

## 4.5.4 准备数据

1: 应该将数据格式化为张量。

2: 这些张量的取值通常应该缩放为较小的值，比如在 [-1, 1] 区间或 [0, 1] 区间。

3: 如果不同的特征具有不同的取值范围（异质数据），那么应该做数据标准化。

4: 可能需要做特征工程，尤其是对于小数据问题。

## 4.5.5 开发比基准更好的模型

你还需要选择三个关键参数来构建第一个工作模型。

1: 最后一层的激活 。它对网络输出进行有效的限制。例如，IMDB 分类的例子在最后一层使用了 sigmoid ，回归的例子在最后一层没有使用激活，等等。

2: 损失函数 。它应该匹配你要解决的问题的类型。例如，IMDB 的例子使用 binary_crossentropy 、回归的例子使用 mse ，等等。

3: 优化配置 。你要使用哪种优化器？学习率是多少？大多数情况下，使用 rmsprop 及其默认的学习率是稳妥的。

关于损失函数的选择，需要注意，直接优化衡量问题成功的指标不一定总是可行的。有时难以将指标转化为损失函数，要知道，损失函数需要在只有小批量数据时即可计算（理想情况下，只有一个数据点时，损失函数应该也是可计算的），而且还必须是可微的（否则无法用反向传播来训练网络）。例如，广泛使用的分类指标 ROC AUC 就不能被直接优化。因此在分类任务中，常见的做法是优化 ROC AUC 的替代指标，比如交叉熵。一般来说，你可以认为交叉熵越小，ROC AUC 越大。

问题类型    ～～      最后一层激活     ～～     损失函数

二分类问题 ～～ sigmoid ～～ binary_crossentropy

多分类、单标签问题 ～～ softmax ～～ categorical_crossentropy

（训练集中的每一个样本只有一个相关的标签。）

多分类、多标签问题 ～～ sigmoid ～～	binary_crossentropy 

（同一个实例，可以有多个标签。）

回归到任意值 ～～ 无 ～～ mse

回归到0-1范围内的值 ～～ sigmoid ～～ mse或者binary_crossentropy

## 4.5.6 扩大模型规模:开发过拟合的模型

要搞清楚你需要多大的模型，就必须开发一个过拟合的模型，这很简单。

(1)添加更多的层.

(2)让每一层变得更大.

(3)训练更多的轮次.

要始终监控训练损失和验证损失，以及你所关心的指标的训练值和验证值。如果你发现模型在验证数据上的性能开始下降，那么就出现了过拟合。

下一阶段将开始正则化和调节模型，以便尽可能地接近理想模型，既不过拟合也不欠拟合。

## 4.5.7 模型正则化与调节超参数

(1)添加 dropout

(2)尝试不同的架构：增加或减少层数

(3)添加 L1 和 / 或 L2 正则化

(4)尝试不同的超参数（比如每层的单元个数或优化器的学习率），以找到最佳配置

(5)（可选）反复做特征工程：添加新特征或删除没有信息量的特征

请注意：每次使用验证过程的反馈来调节模型，都会将有关验证过程的信息泄露到模型中。如果只重复几次，那么无关紧要；但如果系统性地迭代许多次，最终会导致模型对验证过程过拟合（即使模型并没有直接在验证数据上训练）。这会降低验证过程的可靠性。

一旦开发出令人满意的模型配置，你就可以在所有可用数据（训练数据 + 验证数据）上训练最终的生产模型，然后在测试集上最后评估一次。如果测试集上的性能比验证集上差很多，那么这可能意味着你的验证流程不可靠，或者你在调节模型参数时在验证数据上出现了过拟合。在这种情况下，你可能需要换用更加可靠的评估方法，比如重复的 K 折验证。

查看 history 字典

In [1]:
print(original_hist.history.keys())

NameError: name 'original_hist' is not defined

# 第五章:

## 1. 数据预处理

你现在已经知道，将数据输入神经网络之前，应该将数据格式化为经过预处理的浮点数张量。现在，数据以 JPEG 文件的形式保存在硬盘中，所以数据预处理步骤大致如下。

(1) 读取图像文件。

(2) 将 JPEG 文件解码为 RGB 像素网格。

(3) 将这些像素网格转换为浮点数张量。

(4) 将像素值（0~255 范围内）缩放到 [0, 1] 区间（正如你所知，神经网络喜欢处理较小的输入值）。

这些步骤可能看起来有点吓人，但幸运的是，Keras 拥有自动完成这些步骤的工具。Keras 有一个图像处理辅助工具的模块，位于 keras.preprocessing .image。特别地，它包含 ImageDataGenerator 类，可以快速创建 Python 生成器，能够将硬盘上的图像文件自动转换为预处理好的张量批量。下面我们将用到这个类。

## 预处理单张图片 1:

In [3]:
from keras.preprocessing.image import ImageDataGenerator

train_datagen = ImageDataGenerator(rescale=1./255)   （以下2行）将所有图像乘以1/255缩放
test_datagen = ImageDataGenerator(rescale=1./255)    

train_generator = train_datagen.flow_from_directory(
        train_dir,   ←------ 目标目录
        target_size=(150, 150),   ←------ 将所有图像的大小调整为150×150
        batch_size=20,
        class_mode='binary')   ←------ 因为使用了binary_crossentropy损失，所以需要用二进制标签

validation_generator = test_datagen.flow_from_directory(
        validation_dir,
        target_size=(150, 150),
        batch_size=20,
        class_mode='binary')

SyntaxError: invalid character in identifier (<ipython-input-3-3d6662f397b9>, line 3)

## 预处理单张图片 2:

In [2]:
img_path = '/Users/admin/Downloads/cats_and_dogs_small/test/cats/cat.1700.jpg'

# 将图像预处理为一个4D张量
from tensorflow.keras.preprocessing import image
import numpy as np

img = image.load_img(img_path, target_size=(150, 150)) #load img
img_tensor = image.img_to_array(img)  # 将图像转为 array
#print("img_tensor1 = ",img_tensor)

img_tensor = np.expand_dims(img_tensor, axis=0) 
#加了一维度,把维度往右边推了一位
#1 是图片的编号, 150 150 是图片的尺寸,3 是图片的通道数
#print("img_tensor2 = ",img_tensor)

img_tensor /= 255.
#请记住，训练模型的输入数据都用这种方法预处理



# Its shape is (1, 150, 150, 3)
print(img_tensor.shape)

(1, 150, 150, 3)


## 预处理单张图片 3: (重要)

In [4]:
from keras.preprocessing import image
from keras.applications.vgg16 import preprocess_input, decode_predictions
import numpy as np

img_path = '/Users/fchollet/Downloads/creative_commons_elephant.jpg'   ←------ 目标图像的本地路径

img = image.load_img(img_path, target_size=(224, 224))   ←------ 大小为224×224的Python图像库（PIL，Python imaging library）图像

x = image.img_to_array(img)   ←------ 形状为 (224, 224, 3)的float32格式的Numpy数组

x = np.expand_dims(x, axis=0)   ←------ 添加一个维度，将数组转换为(1, 224, 224, 3)形状的批量

x = preprocess_input(x)   ←------ 对批量进行预处理（按通道进行颜色标准化）

SyntaxError: invalid character in identifier (<ipython-input-4-6316c60dedb2>, line 5)

## 2. 数据增强
过拟合的原因是学习样本太少，导致无法训练出能够泛化到新数据的模型。如果拥有无限的数据，那么模型能够观察到数据分布的所有内容，这样就永远不会过拟合。数据增强是从现有的训练样本中生成更多的训练数据，其方法是利用多种能够生成可信图像的随机变换来增加 （augment）样本。其目标是，模型在训练时不会两次查看完全相同的图像。这让模型能够观察到数据的更多内容，从而具有更好的泛化能力。

在 Keras 中，这可以通过对 ImageDataGenerator 实例读取的图像执行多次随机变换来实现。我们先来看一个例子。

keras 的默认模型路径 : 快速搜索~/.keras/models

## 绘制图像标准代码:

In [1]:
import matplotlib.pyplot as plt

acc = history.history['acc']
val_acc = history.history['val_acc']
loss = history.history['loss']
val_loss = history.history['val_loss']

epochs = range(len(acc))

plt.plot(epochs, acc, 'bo', label='Training acc')
plt.plot(epochs, val_acc, 'b', label='Validation acc')
plt.title('Training and validation accuracy')
plt.legend()

plt.figure()

plt.plot(epochs, loss, 'bo', label='Training loss')
plt.plot(epochs, val_loss, 'b', label='Validation loss')
plt.title('Training and validation loss')
plt.legend()

plt.show()

NameError: name 'history' is not defined